<a href="https://colab.research.google.com/github/trimtechnologies/ALL-IDS-DATASETS/blob/main/deep_cnn_with_synthetic_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imblearn
!pip install torch
!pip install pytorch-tabnet
!pip install tensorflow
!pip install keras

In [ ]:
import numpy as np
import pandas as pd
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", UserWarning)

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize

import collections

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import BorderlineSMOTE 
from imblearn.over_sampling import ADASYN

from sklearn.model_selection import train_test_split

#from sklearn import tree

from sklearn.metrics import precision_recall_fscore_support

#from sklearn.ensemble import RandomForestClassifier

#from sklearn.ensemble import AdaBoostClassifier
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
#from pytorch_tabnet.tab_model import TabNetClassifier
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = "/content/drive/MyDrive/2017_Dataset/Multi-Class/all_data_merged_attacks.csv"
read = pd.read_csv(data)
df = read.iloc[:,:]
df.shape

In [ ]:
#df.info()

In [ ]:
#df.columns

In [ ]:
#print(df.iloc[:,:76])

#scaler = MinMaxScaler()

scaler = MaxAbsScaler()

X = scaler.fit_transform(df[['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
       'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
       'Idle Std', 'Idle Max', 'Idle Min']])

#X =  normalize(df.iloc[:,:76])
print(X.shape)
y = df['Label']
print(y.shape)

y = LabelEncoder().fit_transform(y)

In [ ]:
print(collections.Counter(y))

In [ ]:
strategy = {0: 2359289, 4: 252661, 7: 252661, 3: 252661, 2: 252661, 8: 252661, 1: 252661, 6: 252661, 5: 252661}
#{1: 2359289, 0: 1132298}
#{1: 1651502, 0: 792608}
#smt = SMOTE(sampling_strategy=strategy, k_neighbors=5, random_state=42)
#X, y = smt.fit_resample(X, y)

#strategy_tl = {0: 252661, 4: 252661, 7: 252661, 3: 252661, 2: 252661, 8: 252661, 1: 252661, 6: 252661, 5: 252661}
rus = RandomUnderSampler(sampling_strategy='not minority',random_state=42)
X, y = rus.fit_resample(X, y)

In [ ]:
print(collections.Counter(y))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state=42, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size = 0.5, random_state=42, stratify=y_test)

a = np.array(y_train)
b = np.zeros((a.size, a.max()+1))
b[np.arange(a.size),a] = 1
y_train = b
print(y_train.shape)

a = np.array(y_test)
b = np.zeros((a.size, a.max()+1))
b[np.arange(a.size),a] = 1
y_test = b
print(y_test.shape)

a = np.array(y_val)
b = np.zeros((a.size, a.max()+1))
b[np.arange(a.size),a] = 1
y_val = b
print(y_val.shape)

In [ ]:
# reshape the data for CNN
X_train = X_train.reshape(len(X_train), X_train.shape[1], 1)
X_test = X_test.reshape(len(X_test), X_test.shape[1], 1)
X_val = X_val.reshape(len(X_val), X_val.shape[1], 1)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu',input_shape=(76,1)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, verbose=0)

#KerasClassifier(build_fn=model, epochs=200, batch_size=5, verbose=0)



In [ ]:
accuracy = model.evaluate(X_test, y_test)
print((accuracy*100))

In [ ]:
y_pred = model.predict(X_test)

y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_test_class = np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test_class, y_pred, digits=6))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(y_test_class, y_pred).plot()